In [147]:
import requests, json
from bs4 import BeautifulSoup
import pandas as pd

site = requests.get('https://www.leedsgrandmosque.com/')
soup = BeautifulSoup(site.content, 'html.parser')

prayer_section = soup.find(class_="prayers")

prayer = [pr.get_text().capitalize() for pr in prayer_section.select(".row .prayer-name")]
ptime = [t.get_text() for t in prayer_section.select(".row .date")]
jammah = [j.get_text() for j in prayer_section.select(".row .jammah-date")]

Table = pd.DataFrame({
    "Prayer": prayer,
    "Time": ptime,
    "Jammah": jammah,
})

Table


,Prayer,Time,Jammah
0,Fajr,5:28,6:00
1,Shurooq,6:55,
2,Dhuhr,12:58,13:30
3,Asr,16:09,17:00
4,Maghrib,19:00,19:05
5,Isha,20:28,21:00


In [223]:
from datetime import datetime, date
import json

token = "secret_WHlov0KR74x7RfOdTPNKVqKpDmP0qVZr0Lrn5Q1DMtt"
databaseId = "663d4598214844c796394eb2d8f42c46"

headers = {
    "Authorization": "Bearer " + token,
    "Content-Type": "application/json",
    "Notion-Version": "2022-02-22"
}

def readDatabase(databaseId, headers):
    readUrl = f"https://api.notion.com/v1/databases/{databaseId}/query"

    res = requests.request("POST", readUrl, headers=headers)
    data = res.json()
    print(res.status_code)
    # print(res.text)

    with open('./Prayer_Time.json', 'w', encoding='utf8') as f:
        json.dump(data, f, ensure_ascii=False)

def createPage(databaseId, headers, title = ""):
    createUrl = 'https://api.notion.com/v1/pages'

    newPageData = {
        "parent": { "database_id": databaseId },
        "properties": {
            "Prayer": {
                "title": [
                    {
                        "text": {
                            "content": title
                        }
                    }
                ]
            },
            "Time": {
                "date": {
                    "start": str(datetime.strptime(str(date.today()) + " " + ptime[0], '%Y-%m-%d %H:%M')),
                    #"end": "2021-05-17T12:00:00Z"
                    "time_zone": "Europe/London"
                }
            },
            "Jummah": {
                "date": {
                    "start": str(datetime.strptime(str(date.today()) + " " + jammah[0], '%Y-%m-%d %H:%M')),
                    #"end": "2021-05-17T12:00:00Z"
                    "time_zone": "Europe/London"
                }
            }
        }
    }
    
    data = json.dumps(newPageData)
    # print(str(uploadData))

    res = requests.request("POST", createUrl, headers=headers, data=data)

    print(res.status_code)
    print(res.text)

def updatePage(headers):
    data = json.load(open("Prayer_Time.json"))

    pageId = {}
    for i in data["results"]:
        pageId[i["properties"]["Prayer"]["title"][0]["plain_text"]] = i["id"]

    for i in range(len(pageId)):
        updateUrl = f"https://api.notion.com/v1/pages/{pageId[i]}"
        try:
            updateData = {
                "properties": {
                    "Time": {
                        "date": {
                            "start": str(datetime.strptime(str(date.today()) + " " + ptime[i], '%Y-%m-%d %H:%M')),
                            #"end": "2021-05-17T12:00:00Z"
                            "time_zone": "Europe/London"
                        }
                    },
                    "Jummah": {
                        "date": {
                            "start": str(datetime.strptime(str(date.today()) + " " + jammah[i], '%Y-%m-%d %H:%M')),
                            #"end": "2021-05-17T12:00:00Z"
                            "time_zone": "Europe/London"
                        }
                    }
                }
            }
        except:
            updateData = {
                "properties": {
                    "Time": {
                        "date": {
                            "start": str(datetime.strptime(str(date.today()) + " " + ptime[i], '%Y-%m-%d %H:%M')),
                            #"end": "2021-05-17T12:00:00Z"
                            "time_zone": "Europe/London"
                        }
                    },
                    "Jummah": {
                        "date": None
                    }
                }
            }

        data = json.dumps(updateData)

        response = requests.request("PATCH", updateUrl, headers=headers, data=data)

    print(response.status_code)
    print(response.text)

# readDatabase(databaseId, headers)
# createPage(databaseId, headers, "Fajr")
updatePage(headers)


KeyError: 0

In [222]:
data = json.load(open("Prayer_Time.json"))
pageId = list()
for i in data["results"]:
    pageId[i["properties"]["Prayer"]["title"][0]["plain_text"]] = i["id"]
print(pageId)
print(ptime)
print(jammah)

{'Asr': '1247990c-ffcd-42af-8d67-140fa8c02fb2', 'Isha': 'bcabb244-f0bb-401a-9db0-0a07f52266d5', 'Maghrib': 'cffab312-0afb-4a51-9ae6-6bccdb01de4e', 'Dhuhr': '1834c13b-62c5-4434-b1d7-1292479bb23a', 'Shurooq': '9f14c9f7-8274-4861-8b5b-d4e855c08b34', 'Fajr': 'd74b5a76-9bdd-4516-8ef2-509b41d94c83'}
['5:28', '6:55', '12:58', '16:09', '19:00', '20:28']
['6:00', '', '13:30', '17:00', '19:05', '21:00']
